In [1]:
pip install transformers==4.3.2

In [2]:
pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

In [7]:
labels = ["Positive", "Negative", "Neutral"]
index_to_labels = {index: label for index, label in enumerate(labels)}

In [4]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

In [8]:
# making response body
class Prediction(BaseModel):
  data: str


app = FastAPI()

@app.get('/')
async def home():
  return "please go to /docs"

# path used for predicting after uploading image file
@app.post('/predict/')
async def predict_out(data):
  model_name = 'distilbert-base-uncased'
  config = AutoConfig.from_pretrained(model_name, num_labels=3)
  model = AutoModelForSequenceClassification.from_config(config=config)
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
  data = data
  #y = data['data']
  prepared_input = tokenizer.prepare_seq2seq_batch([data], return_tensors='pt')
  model = model.to('cpu')
  model.eval()
  model_output = model(**prepared_input)
  prediction = np.argmax(model_output.logits[0].detach().numpy())
  text = index_to_labels[prediction]
 
  return {
      'predict_class': text
 
  }
  

In [9]:
# using ngrok on google colab to expose url so it can be accessed from browser on my PC
ngrok_tunnel = ngrok.connect(7000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=7000)

Public URL: http://ca8fe0d71a54.ngrok.io


INFO:     Started server process [246]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7000 (Press CTRL+C to quit)


INFO:     45.116.232.62:0 - "GET / HTTP/1.1" 200 OK
INFO:     45.116.232.62:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
